In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/train_data.csv')
test = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/test_data.csv')
train_labels=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/train_labels.csv')
fitness=pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/fitness_values.csv')


In [2]:
list=(['patient_id', 'event_name','specialty','plan_type'])
    
for x in train.columns:
        if x in list:
            train[x]=train[x].astype(np.object)
for x in test.columns:
        if x in list:
            test[x]=test[x].astype(np.object)

In [3]:
print(train.shape)
train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

print(train.shape)
print(test.shape)
test.head()

(14446880, 6)
(7171012, 6)
(3083639, 6)


,patient_id,event_name,event_time,specialty,plan_type,patient_payment
0,patient_2,event_1,164,spec_1,plan_2,0.0
1,patient_3,event_1,420,spec_1,plan_2,0.0
2,patient_5,event_1,623,spec_1,plan_2,0.0
3,patient_8,event_1,162,spec_1,plan_1,0.0
4,patient_14,event_1,243,spec_1,plan_2,0.0


In [9]:
#print(test[test['patient_id']=='patient_22688'])

In [10]:
#print(test[test['patient_id']=='patient_22688'])

In [4]:
train['flag']='train'
test['flag']='test'

train.head()

,patient_id,event_name,event_time,specialty,plan_type,patient_payment,flag
0,patient_1,event_1,47,spec_1,plan_1,0.0,train
1,patient_4,event_1,727,spec_1,plan_2,0.0,train
2,patient_6,event_1,738,spec_1,plan_1,0.0,train
3,patient_7,event_1,923,spec_1,plan_1,0.0,train
4,patient_9,event_1,49,spec_1,plan_1,0.0,train


In [5]:
combo=train.append(test,sort=False)
combo.shape
combo.head()
combo['patient_id'].value_counts()
combo.shape


(10254651, 7)

In [26]:
combo_event_flag=combo
combo_event_flag['event_flag']=1
combo_event_pivot=combo.pivot_table(index=['patient_id'], columns='event_name', values=('event_flag'), aggfunc='first',fill_value=0).reset_index()
print(combo_event_pivot.head())
print(combo_event_pivot.shape)


event_name     patient_id  event_1  event_10  event_100  event_101  event_102  \
0               patient_1        1         1          0          1          0   
1              patient_10        1         1          0          1          0   
2             patient_100        1         0          0          0          0   
3            patient_1000        1         0          0          1          0   
4           patient_10000        0         1          0          0          0   

event_name  event_103  event_104  event_105  event_106  ...  event_90  \
0                   0          1          1          0  ...         1   
1                   0          1          1          0  ...         0   
2                   0          0          1          0  ...         0   
3                   0          0          1          0  ...         1   
4                   0          1          1          1  ...         0   

event_name  event_91  event_92  event_93  event_94  event_95  event_96  \


In [6]:
combo_count_pos_pays=combo[combo['patient_payment'] > 0].groupby('patient_id')['patient_payment'].count().sort_values(ascending=False).reset_index()
combo_count_number_pays=combo.groupby('patient_id')['patient_payment'].count().sort_values(ascending=False).reset_index()

combo_counts_unique_event=combo.groupby('patient_id')['event_name'].nunique().sort_values(ascending=False).reset_index()
combo_counts_unique_spec=combo.groupby('patient_id')['specialty'].nunique().sort_values(ascending=False).reset_index()
combo_counts_unique_plan=combo.groupby('patient_id')['plan_type'].nunique().sort_values(ascending=False).reset_index()

combo_max_pay=combo.groupby('patient_id')['patient_payment'].max().sort_values(ascending=False).reset_index()

combo_max_time=combo.groupby('patient_id')['event_time'].max().sort_values(ascending=False).reset_index()
combo_min_time=combo.groupby('patient_id')['event_time'].min().sort_values(ascending=False).reset_index()
combo_mean_time=combo.groupby('patient_id')['event_time'].mean().sort_values(ascending=False).reset_index()

In [7]:
print(combo_count_pos_pays.shape)
print(combo_count_number_pays.shape)
print(combo_counts_unique_event.shape)
print(combo_counts_unique_spec.shape)
print(combo_counts_unique_plan.shape)
print(combo_max_pay.shape)
print(combo_max_time.shape)
print(combo_min_time.shape)
print(combo_mean_time.shape)

dfs=([combo_count_pos_pays,combo_count_number_pays,combo_counts_unique_event,combo_counts_unique_spec,combo_counts_unique_plan,combo_max_pay,combo_max_time,combo_min_time,combo_mean_time])

from functools import reduce
combo_counts = reduce(lambda  left,right: pd.merge(left,right,on=['patient_id'],
                                            how='outer'), dfs).fillna(0) 

combo_counts.head()

(23438, 2)
(23831, 2)
(23831, 2)
(23831, 2)
(23831, 2)
(23831, 2)
(23831, 2)
(23831, 2)
(23831, 2)


,patient_id,patient_payment_x,patient_payment_y,event_name,specialty,plan_type,patient_payment,event_time_x,event_time_y,event_time
0,patient_1075,1390.0,3335,118,6,3,322.157143,1079,1,679.029385
1,patient_55,1311.0,2835,129,19,3,250.000000,1075,7,541.945679
2,patient_2684,1251.0,2222,96,7,3,250.000000,1074,1,439.319082
3,patient_4900,1232.0,2243,133,16,3,250.000000,1080,3,553.317432
4,patient_3197,1216.0,2066,125,14,3,250.000000,1078,1,462.323814


In [8]:
combo_counts.rename(columns={'patient_payment_x':'count_pos_pays','patient_payment_y':'count_rows','event_name':'count_unique_events','specialty':'count_unique_specs','plan_type':'count_unique_plans','patient_payment':'combo_max_pay','event_time_x':'time_max','event_time_y':'time_min','event_time':'time_mean'}, inplace=True)
combo_counts['paid_rows_proportion']=combo_counts['count_pos_pays']/combo_counts['count_rows']
combo_counts.head()
    

,patient_id,count_pos_pays,count_rows,count_unique_events,count_unique_specs,count_unique_plans,combo_max_pay,time_max,time_min,time_mean,paid_rows_proportion
0,patient_1075,1390.0,3335,118,6,3,322.157143,1079,1,679.029385,0.416792
1,patient_55,1311.0,2835,129,19,3,250.000000,1075,7,541.945679,0.462434
2,patient_2684,1251.0,2222,96,7,3,250.000000,1074,1,439.319082,0.563006
3,patient_4900,1232.0,2243,133,16,3,250.000000,1080,3,553.317432,0.549264
4,patient_3197,1216.0,2066,125,14,3,250.000000,1078,1,462.323814,0.588577


In [9]:
#combo_count=combo.groupby('patient_id')['event_name'].count().sort_values(ascending=False).reset_index()
#combo_count_unique=combo.groupby('patient_id')['event_name'].nunique().sort_values(ascending=False).reset_index()

#combo_filtered= combo_count.iloc[0:200,0].reset_index()

#combo_count_unique.head()
#combo.shape

In [10]:
#combo_filtered_excel=pd.merge(combo_filtered,combo,left_on='patient_id',right_on='patient_id',how='left')
#combo_filtered_excel
#combo_filtered_excel.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/filtered rows for analysis',index=False)


In [27]:
combo_pivot = combo.pivot_table(index=['patient_id','flag'], columns='event_name', values=('event_time','patient_payment'), aggfunc=[np.min,np.max,np.mean,np.sum])
combo_pivot.drop([['mean','patient_payment'],['amin','patient_payment'],['amax','patient_payment'],['sum','event_time']],axis=1,inplace=True)
combo_pivot.shape



(23831, 2080)

In [12]:
#combo_categ_sort = combo.groupby(['patient_id', 'event_name'], as_index=False)['specialty','plan_type'].count()\
#        .sort_values(['specialty','plan_type'], ascending=False)

combo_categ_sort_spec=combo.groupby(['patient_id', 'event_name'], as_index=False)['specialty'].max()
combo_categ_sort_plan=combo.groupby(['patient_id', 'event_name'], as_index=False)['plan_type'].max()




In [13]:
#combo_categ_sort.drop_duplicates(keep=False,inplace=True)
combo_categ_sort=pd.merge(combo_categ_sort_spec,combo_categ_sort_plan,left_on=['patient_id','event_name'],right_on=['patient_id','event_name'])
combo_categ_sort
#combo_dump= combo[combo['patient_id']=='patient_1']
#combo_dump.append(combo[combo['patient_id']=='patient_1'])
#combo_dump.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/highest_freq_test',index=False)
#combo_categ_sort
#print(combo_dump[combo_dump['event_name']=='event_68'])

,patient_id,event_name,specialty,plan_type
0,patient_1,event_1,spec_1,plan_1
1,patient_1,event_10,spec_1,plan_1
2,patient_1,event_101,spec_1,plan_1
3,patient_1,event_104,spec_1,plan_1
4,patient_1,event_105,spec_1,plan_1
...,...,...,...,...
1430159,patient_9999,event_71,spec_1,plan_1
1430160,patient_9999,event_82,spec_1,plan_1
1430161,patient_9999,event_85,spec_1,plan_1
1430162,patient_9999,event_95,spec_1,plan_1


In [14]:
combo_pivot_categ = combo_categ_sort.pivot_table(index=['patient_id'], columns='event_name', values=('specialty','plan_type'),aggfunc='first',fill_value='DNO')
combo_pivot_categ.drop_duplicates(keep=False,inplace=True)
combo_pivot_categ.reset_index(inplace=True)
combo_pivot_categ

patient_id plan_type                                         \
event_name                  event_1 event_10 event_100 event_101 event_102   
0               patient_1    plan_1   plan_1       DNO    plan_1       DNO   
1              patient_10    plan_1   plan_1       DNO    plan_1       DNO   
2             patient_100    plan_1      DNO       DNO       DNO       DNO   
3            patient_1000    plan_1      DNO       DNO    plan_1       DNO   
4           patient_10000       DNO   plan_1       DNO       DNO       DNO   
...                   ...       ...      ...       ...       ...       ...   
23826        patient_9995       DNO      DNO       DNO       DNO       DNO   
23827        patient_9996       DNO   plan_2    plan_4    plan_2       DNO   
23828        patient_9997       DNO      DNO       DNO       DNO       DNO   
23829        patient_9998    plan_2      DNO       DNO       DNO       DNO   
23830        patient_9999       DNO      DNO       DNO    plan_1       DNO   

                                                    ... specialty           \
event_name event_103 event_104 event_105 event_106  ...  event_90 event_91   
0                DNO    plan_1    plan_1       DNO  ...    spec_1      DNO   
1                DNO    plan_1    plan_1       DNO  ...       DNO      DNO   
2                DNO       DNO    plan_1       DNO  ...       DNO      DNO   
3                DNO       DNO    plan_1       DNO  ...    spec_1      DNO   
4                DNO    plan_1    plan_1    plan_1  ...       DNO      DNO   
...              ...       ...       ...       ...  ...       ...      ...   
23826            DNO       DNO    plan_1       DNO  ...    spec_1      DNO   
23827            DNO       DNO    plan_2       DNO  ...       DNO      DNO   
23828            DNO       DNO    plan_2       DNO  ...    spec_1      DNO   
23829            DNO       DNO       DNO       DNO  ...       DNO      DNO   
23830            DNO       DNO    plan_1       DNO  ...       DNO      DNO   

                                                                           \
event_name event_92 event_93 event_94 event_95 event_96 event_97 event_98   
0               DNO      DNO      DNO      DNO      DNO      DNO      DNO   
1               DNO   spec_1      DNO      DNO      DNO      DNO      DNO   
2               DNO      DNO      DNO      DNO   spec_3      DNO      DNO   
3               DNO      DNO      DNO      DNO  spec_12      DNO      DNO   
4               DNO      DNO      DNO      DNO      DNO   spec_6   spec_1   
...             ...      ...      ...      ...      ...      ...      ...   
23826           DNO      DNO      DNO      DNO      DNO      DNO      DNO   
23827           DNO      DNO      DNO      DNO   spec_7      DNO      DNO   
23828           DNO      DNO      DNO      DNO      DNO   spec_3      DNO   
23829           DNO      DNO      DNO      DNO      DNO      DNO      DNO   
23830           DNO      DNO      DNO   spec_1      DNO      DNO   spec_1   

                     
event_name event_99  
0            spec_1  
1            spec_1  
2            spec_1  
3               DNO  
4            spec_1  
...             ...  
23826        spec_1  
23827        spec_1  
23828           DNO  
23829           DNO  
23830           DNO  

[23831 rows x 1041 columns]

In [15]:
combo_pivot_categ

patient_id plan_type                                         \
event_name                  event_1 event_10 event_100 event_101 event_102   
0               patient_1    plan_1   plan_1       DNO    plan_1       DNO   
1              patient_10    plan_1   plan_1       DNO    plan_1       DNO   
2             patient_100    plan_1      DNO       DNO       DNO       DNO   
3            patient_1000    plan_1      DNO       DNO    plan_1       DNO   
4           patient_10000       DNO   plan_1       DNO       DNO       DNO   
...                   ...       ...      ...       ...       ...       ...   
23826        patient_9995       DNO      DNO       DNO       DNO       DNO   
23827        patient_9996       DNO   plan_2    plan_4    plan_2       DNO   
23828        patient_9997       DNO      DNO       DNO       DNO       DNO   
23829        patient_9998    plan_2      DNO       DNO       DNO       DNO   
23830        patient_9999       DNO      DNO       DNO    plan_1       DNO   

                                                    ... specialty           \
event_name event_103 event_104 event_105 event_106  ...  event_90 event_91   
0                DNO    plan_1    plan_1       DNO  ...    spec_1      DNO   
1                DNO    plan_1    plan_1       DNO  ...       DNO      DNO   
2                DNO       DNO    plan_1       DNO  ...       DNO      DNO   
3                DNO       DNO    plan_1       DNO  ...    spec_1      DNO   
4                DNO    plan_1    plan_1    plan_1  ...       DNO      DNO   
...              ...       ...       ...       ...  ...       ...      ...   
23826            DNO       DNO    plan_1       DNO  ...    spec_1      DNO   
23827            DNO       DNO    plan_2       DNO  ...       DNO      DNO   
23828            DNO       DNO    plan_2       DNO  ...    spec_1      DNO   
23829            DNO       DNO       DNO       DNO  ...       DNO      DNO   
23830            DNO       DNO    plan_1       DNO  ...       DNO      DNO   

                                                                           \
event_name event_92 event_93 event_94 event_95 event_96 event_97 event_98   
0               DNO      DNO      DNO      DNO      DNO      DNO      DNO   
1               DNO   spec_1      DNO      DNO      DNO      DNO      DNO   
2               DNO      DNO      DNO      DNO   spec_3      DNO      DNO   
3               DNO      DNO      DNO      DNO  spec_12      DNO      DNO   
4               DNO      DNO      DNO      DNO      DNO   spec_6   spec_1   
...             ...      ...      ...      ...      ...      ...      ...   
23826           DNO      DNO      DNO      DNO      DNO      DNO      DNO   
23827           DNO      DNO      DNO      DNO   spec_7      DNO      DNO   
23828           DNO      DNO      DNO      DNO      DNO   spec_3      DNO   
23829           DNO      DNO      DNO      DNO      DNO      DNO      DNO   
23830           DNO      DNO      DNO   spec_1      DNO      DNO   spec_1   

                     
event_name event_99  
0            spec_1  
1            spec_1  
2            spec_1  
3               DNO  
4            spec_1  
...             ...  
23826        spec_1  
23827        spec_1  
23828           DNO  
23829           DNO  
23830           DNO  

[23831 rows x 1041 columns]

In [16]:
combo_categ_sort_spec=combo.groupby(['patient_id'])['specialty'].max().reset_index()
combo_categ_sort_plan=combo.groupby(['patient_id'])['plan_type'].max().reset_index()
combo_categ_sort=pd.merge(combo_categ_sort_spec,combo_categ_sort_plan,left_on=['patient_id'],right_on=['patient_id'])

In [28]:
combo_pivot.reset_index(inplace=True)
combo_pivot.columns = [' '.join(col).strip() for col in combo_pivot.columns.values]
combo_pivot_categ.columns = [' '.join(col).strip() for col in combo_pivot_categ.columns.values]


combo_pivot=pd.merge(combo_pivot,combo_pivot_categ,left_on='patient_id',right_on='patient_id')
combo_pivot=pd.merge(combo_pivot,combo_counts,left_on='patient_id',right_on='patient_id')
combo_pivot=pd.merge(combo_pivot,combo_categ_sort,left_on='patient_id',right_on='patient_id')
combo_pivot=pd.merge(combo_pivot,combo_event_pivot,left_on=['patient_id'],right_on=['patient_id'])


print(combo_pivot.shape)
combo_pivot

KeyError: 'patient_id'

In [ ]:
train=combo_pivot[combo_pivot['flag']=='train']
test=combo_pivot[combo_pivot['flag']=='test']

print(test.shape)
print(train.shape)


In [19]:
#train.columns = [' '.join(col).strip() for col in train.columns.values]
#test.columns=[' '.join(col).strip() for col in test.columns.values]

In [ ]:
train=pd.merge(train,train_labels,left_on='patient_id',right_on='patient_id')
#train['outcome_flag']=train.outcome_flag.apply(lambda x: 'Yes' if x == 1 else 'No')
train.head()

In [ ]:
print(test.shape)
print(train.shape)
train.drop('flag',axis=1,inplace=True)
test.drop('flag',axis=1,inplace=True)
print(test.shape)
print(train.shape)

In [60]:
#train.event_name_order.nunique()
#, values =['patient_id','event_name', 'event_time', 'specialty', 'plan_type',
#       'patient_payment', 'outcome_flag', 'event_order']

#train_trial=train
#print(train_trial.shape)
#train_trial.drop_duplicates(keep=False,inplace=True)
#print(train_trial.shape)

#train_trial=train_trial[['patient_id','event_name_order','event_time']]
#train_trial['key']=train_trial['patient_id']+train_trial['event_name_order']+train_trial['event_time'].map(str)

#train_trial.drop_duplicates(keep=False,inplace=True)
#train_trial.shape

In [59]:
#train.drop('event_name',axis=1,inplace=True)
#train.head()

In [ ]:
#train=train.unstack()
#train_pivot=train_trial.pivot(index ='patient_id', columns ='event_name_order') 
#train_pivot = train.pivot_table(index='patient_id', columns='event_name_order', values=('event_time','specialty','plan_type','patient_payment'), aggfunc=np.min)

In [29]:
#train.describe(include='all')
test.shape

(7148, 3133)

In [58]:
#train_pivot.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/pivot',index=False)

In [21]:
#Alternate code for gridsearch with simple test_trainsplit

Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
params = {'n_estimators': [700,1300],
        'reg_lambda' : [3,13],
        'learning_rate' : [0.3,0.01,0.09],
        'depth' : [5,10],
        'bagging_temperature' : [0]
        }
                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
#from sklearn.model_selection import KFold
#kf = KFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

X_train,X_val,y_train,y_val = train_test_split(Xd,yd,test_size=0.25,random_state = 13)
categorical_features_indices = np.where(Xd.dtypes =='object')[0]
catb = CatBoostRegressor(X_train.values,y_train.values,cat_features=categorical_features_indices,random_seed=13)

#skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 13)

grid = GridSearchCV(estimator=catb, param_grid=params,cv=(X_val, y_val), scoring='roc_auc', n_jobs=-1, verbose=100 )
grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/Tuning_Catreg_pivot_aggr_v1', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

Fitting 2 folds for each of 24 candidates, totalling 48 fits


ValueError: too many values to unpack (expected 2)

In [18]:
Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
params = {'n_estimators': [700,1300],
        'reg_lambda' : [3,13],
        'learning_rate' : [0.3,0.01,0.09],
        'depth' : [5,10],
        'bagging_temperature' : [0]
        }

In [23]:
#Alternate code for gridsearch with stratkfold

Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
params = {'n_estimators': [700,1000],
        'reg_lambda' : [4,9],
        'learning_rate' : [0.07,0.13],
        'depth' : [4,7],
        'bagging_temperature' : [1]
        }                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

categorical_features_indices = np.where(Xd.dtypes =='object')[0]
catb = CatBoostRegressor(cat_features=categorical_features_indices,random_seed=13,early_stopping_rounds=100)

skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 13)
print('start_gridsearch')
grid = GridSearchCV(estimator=catb, param_grid=params, cv=skf.split(Xd,yd),verbose=100)
print('start_fit')
grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/Full_Tuning_output_Catreg_pivot_aggr_new_features_v2', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

start_gridsearch
start_fit
Fitting 3 folds for each of 16 candidates, totalling 48 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] bagging_temperature=1, depth=4, learning_rate=0.07, n_estimators=700, reg_lambda=4 
0:	learn: 0.3509459	total: 615ms	remaining: 7m 9s
1:	learn: 0.3484482	total: 1.2s	remaining: 6m 59s
2:	learn: 0.3468629	total: 1.71s	remaining: 6m 36s
3:	learn: 0.3453421	total: 2.24s	remaining: 6m 29s
4:	learn: 0.3438105	total: 2.78s	remaining: 6m 26s
5:	learn: 0.3425649	total: 3.29s	remaining: 6m 20s
6:	learn: 0.3412452	total: 3.81s	remaining: 6m 17s
7:	learn: 0.3397288	total: 4.37s	remaining: 6m 18s
8:	learn: 0.3384937	total: 4.88s	remaining: 6m 14s
9:	learn: 0.3373980	total: 5.4s	remaining: 6m 12s
10:	learn: 0.3366812	total: 5.93s	remaining: 6m 11s
11:	learn: 0.3360571	total: 6.49s	remaining: 6m 12s
12:	learn: 0.3353080	total: 7.06s	remaining: 6m 13s
13:	learn: 0.3345951	total: 7.65s	remaining: 6m 14s
14:	learn: 0.3339200	total:

152:	learn: 0.3119277	total: 1m 29s	remaining: 5m 18s
153:	learn: 0.3118629	total: 1m 29s	remaining: 5m 17s
154:	learn: 0.3117379	total: 1m 30s	remaining: 5m 16s
155:	learn: 0.3116995	total: 1m 30s	remaining: 5m 16s
156:	learn: 0.3116132	total: 1m 31s	remaining: 5m 15s
157:	learn: 0.3114989	total: 1m 31s	remaining: 5m 15s
158:	learn: 0.3113287	total: 1m 32s	remaining: 5m 15s
159:	learn: 0.3111485	total: 1m 33s	remaining: 5m 14s
160:	learn: 0.3111270	total: 1m 33s	remaining: 5m 14s
161:	learn: 0.3109612	total: 1m 34s	remaining: 5m 13s
162:	learn: 0.3109344	total: 1m 35s	remaining: 5m 13s
163:	learn: 0.3107021	total: 1m 35s	remaining: 5m 12s
164:	learn: 0.3106297	total: 1m 36s	remaining: 5m 12s
165:	learn: 0.3103677	total: 1m 36s	remaining: 5m 11s
166:	learn: 0.3102950	total: 1m 37s	remaining: 5m 11s
167:	learn: 0.3100844	total: 1m 38s	remaining: 5m 10s
168:	learn: 0.3099669	total: 1m 38s	remaining: 5m 9s
169:	learn: 0.3099282	total: 1m 39s	remaining: 5m 9s
170:	learn: 0.3097069	total: 1

305:	learn: 0.2996273	total: 2m 59s	remaining: 3m 51s
306:	learn: 0.2995757	total: 3m	remaining: 3m 51s
307:	learn: 0.2995681	total: 3m 1s	remaining: 3m 50s
308:	learn: 0.2995333	total: 3m 1s	remaining: 3m 49s
309:	learn: 0.2994093	total: 3m 2s	remaining: 3m 49s
310:	learn: 0.2993304	total: 3m 2s	remaining: 3m 48s
311:	learn: 0.2993114	total: 3m 3s	remaining: 3m 48s
312:	learn: 0.2991312	total: 3m 3s	remaining: 3m 47s
313:	learn: 0.2990656	total: 3m 4s	remaining: 3m 46s
314:	learn: 0.2989339	total: 3m 5s	remaining: 3m 46s
315:	learn: 0.2989262	total: 3m 5s	remaining: 3m 45s
316:	learn: 0.2988196	total: 3m 6s	remaining: 3m 45s
317:	learn: 0.2988164	total: 3m 6s	remaining: 3m 44s
318:	learn: 0.2987607	total: 3m 7s	remaining: 3m 43s
319:	learn: 0.2986507	total: 3m 8s	remaining: 3m 43s
320:	learn: 0.2986422	total: 3m 8s	remaining: 3m 42s
321:	learn: 0.2985008	total: 3m 9s	remaining: 3m 42s
322:	learn: 0.2984258	total: 3m 10s	remaining: 3m 41s
323:	learn: 0.2982920	total: 3m 10s	remaining: 

458:	learn: 0.2903349	total: 4m 31s	remaining: 2m 22s
459:	learn: 0.2903280	total: 4m 32s	remaining: 2m 21s
460:	learn: 0.2901423	total: 4m 32s	remaining: 2m 21s
461:	learn: 0.2901154	total: 4m 33s	remaining: 2m 20s
462:	learn: 0.2899964	total: 4m 33s	remaining: 2m 20s
463:	learn: 0.2899282	total: 4m 34s	remaining: 2m 19s
464:	learn: 0.2899166	total: 4m 34s	remaining: 2m 18s
465:	learn: 0.2898085	total: 4m 35s	remaining: 2m 18s
466:	learn: 0.2897731	total: 4m 36s	remaining: 2m 17s
467:	learn: 0.2897017	total: 4m 36s	remaining: 2m 17s
468:	learn: 0.2896089	total: 4m 37s	remaining: 2m 16s
469:	learn: 0.2895119	total: 4m 37s	remaining: 2m 15s
470:	learn: 0.2894301	total: 4m 38s	remaining: 2m 15s
471:	learn: 0.2893999	total: 4m 38s	remaining: 2m 14s
472:	learn: 0.2893190	total: 4m 39s	remaining: 2m 14s
473:	learn: 0.2891605	total: 4m 39s	remaining: 2m 13s
474:	learn: 0.2891365	total: 4m 40s	remaining: 2m 12s
475:	learn: 0.2891299	total: 4m 41s	remaining: 2m 12s
476:	learn: 0.2890903	total:

612:	learn: 0.2831517	total: 6m 4s	remaining: 51.8s
613:	learn: 0.2830873	total: 6m 5s	remaining: 51.2s
614:	learn: 0.2830752	total: 6m 6s	remaining: 50.6s
615:	learn: 0.2830560	total: 6m 6s	remaining: 50s
616:	learn: 0.2829191	total: 6m 7s	remaining: 49.4s
617:	learn: 0.2829101	total: 6m 8s	remaining: 48.8s
618:	learn: 0.2829081	total: 6m 8s	remaining: 48.2s
619:	learn: 0.2829016	total: 6m 9s	remaining: 47.7s
620:	learn: 0.2828375	total: 6m 9s	remaining: 47.1s
621:	learn: 0.2827940	total: 6m 10s	remaining: 46.5s
622:	learn: 0.2827868	total: 6m 11s	remaining: 45.9s
623:	learn: 0.2827401	total: 6m 11s	remaining: 45.3s
624:	learn: 0.2827284	total: 6m 12s	remaining: 44.7s
625:	learn: 0.2826904	total: 6m 12s	remaining: 44.1s
626:	learn: 0.2826713	total: 6m 13s	remaining: 43.5s
627:	learn: 0.2826150	total: 6m 14s	remaining: 42.9s
628:	learn: 0.2826110	total: 6m 14s	remaining: 42.3s
629:	learn: 0.2825613	total: 6m 15s	remaining: 41.7s
630:	learn: 0.2825245	total: 6m 16s	remaining: 41.1s
631:

...

392:	learn: 0.2759448	total: 15m 35s	remaining: 24m 5s
393:	learn: 0.2758732	total: 15m 38s	remaining: 24m 2s
394:	learn: 0.2757802	total: 15m 40s	remaining: 24m
395:	learn: 0.2756949	total: 15m 42s	remaining: 23m 57s
396:	learn: 0.2756199	total: 15m 45s	remaining: 23m 55s
397:	learn: 0.2756084	total: 15m 47s	remaining: 23m 53s
398:	learn: 0.2754038	total: 15m 49s	remaining: 23m 50s
399:	learn: 0.2752861	total: 15m 51s	remaining: 23m 47s
400:	learn: 0.2751801	total: 15m 53s	remaining: 23m 44s
401:	learn: 0.2751642	total: 15m 56s	remaining: 23m 42s
402:	learn: 0.2750659	total: 15m 58s	remaining: 23m 40s
403:	learn: 0.2749850	total: 16m	remaining: 23m 37s
404:	learn: 0.2749536	total: 16m 3s	remaining: 23m 35s
405:	learn: 0.2748544	total: 16m 5s	remaining: 23m 32s
406:	learn: 0.2747950	total: 16m 7s	remaining: 23m 30s
407:	learn: 0.2747749	total: 16m 10s	remaining: 23m 27s
408:	learn: 0.2745862	total: 16m 12s	remaining: 23m 24s
409:	learn: 0.2745738	total: 16m 14s	remaining: 23m 22s
410:	

540:	learn: 0.2653360	total: 21m 15s	remaining: 18m 2s
541:	learn: 0.2652725	total: 21m 18s	remaining: 17m 59s
542:	learn: 0.2652151	total: 21m 20s	remaining: 17m 57s
543:	learn: 0.2651079	total: 21m 23s	remaining: 17m 55s
544:	learn: 0.2650617	total: 21m 25s	remaining: 17m 53s
545:	learn: 0.2649557	total: 21m 27s	remaining: 17m 50s
546:	learn: 0.2649362	total: 21m 30s	remaining: 17m 48s
547:	learn: 0.2648461	total: 21m 32s	remaining: 17m 45s
548:	learn: 0.2647942	total: 21m 34s	remaining: 17m 43s
549:	learn: 0.2647818	total: 21m 38s	remaining: 17m 42s
550:	learn: 0.2647652	total: 21m 40s	remaining: 17m 39s
551:	learn: 0.2647551	total: 21m 42s	remaining: 17m 37s
552:	learn: 0.2645590	total: 21m 44s	remaining: 17m 34s
553:	learn: 0.2645547	total: 21m 47s	remaining: 17m 32s
554:	learn: 0.2644481	total: 21m 49s	remaining: 17m 30s
555:	learn: 0.2644405	total: 21m 51s	remaining: 17m 27s
556:	learn: 0.2643525	total: 21m 54s	remaining: 17m 25s
557:	learn: 0.2642645	total: 21m 57s	remaining: 1

688:	learn: 0.2553863	total: 26m 55s	remaining: 12m 9s
689:	learn: 0.2553031	total: 26m 57s	remaining: 12m 6s
690:	learn: 0.2552359	total: 27m	remaining: 12m 4s
691:	learn: 0.2552242	total: 27m 2s	remaining: 12m 2s
692:	learn: 0.2552204	total: 27m 5s	remaining: 11m 59s
693:	learn: 0.2551438	total: 27m 7s	remaining: 11m 57s
694:	learn: 0.2551021	total: 27m 9s	remaining: 11m 55s
695:	learn: 0.2550811	total: 27m 11s	remaining: 11m 52s
696:	learn: 0.2549715	total: 27m 13s	remaining: 11m 50s
697:	learn: 0.2549674	total: 27m 15s	remaining: 11m 47s
698:	learn: 0.2549296	total: 27m 18s	remaining: 11m 45s
699:	learn: 0.2548650	total: 27m 20s	remaining: 11m 43s
700:	learn: 0.2548014	total: 27m 22s	remaining: 11m 40s
701:	learn: 0.2547355	total: 27m 24s	remaining: 11m 38s
702:	learn: 0.2546265	total: 27m 26s	remaining: 11m 35s
703:	learn: 0.2545214	total: 27m 29s	remaining: 11m 33s
704:	learn: 0.2544777	total: 27m 31s	remaining: 11m 31s
705:	learn: 0.2544671	total: 27m 34s	remaining: 11m 28s
706:

838:	learn: 0.2474431	total: 33m 23s	remaining: 6m 24s
839:	learn: 0.2473563	total: 33m 26s	remaining: 6m 22s
840:	learn: 0.2473553	total: 33m 27s	remaining: 6m 19s
841:	learn: 0.2472864	total: 33m 30s	remaining: 6m 17s
842:	learn: 0.2472548	total: 33m 32s	remaining: 6m 14s
843:	learn: 0.2472073	total: 33m 35s	remaining: 6m 12s
844:	learn: 0.2471422	total: 33m 38s	remaining: 6m 10s
845:	learn: 0.2470712	total: 33m 40s	remaining: 6m 7s
846:	learn: 0.2469505	total: 33m 43s	remaining: 6m 5s
847:	learn: 0.2468989	total: 33m 46s	remaining: 6m 3s
848:	learn: 0.2468812	total: 33m 48s	remaining: 6m
849:	learn: 0.2467987	total: 33m 51s	remaining: 5m 58s
850:	learn: 0.2467912	total: 33m 53s	remaining: 5m 56s
851:	learn: 0.2467726	total: 33m 56s	remaining: 5m 53s
852:	learn: 0.2467071	total: 33m 59s	remaining: 5m 51s
853:	learn: 0.2466990	total: 34m 1s	remaining: 5m 49s
854:	learn: 0.2466467	total: 34m 4s	remaining: 5m 46s
855:	learn: 0.2466407	total: 34m 7s	remaining: 5m 44s
856:	learn: 0.246564

989:	learn: 0.2399532	total: 39m 27s	remaining: 23.9s
990:	learn: 0.2399458	total: 39m 29s	remaining: 21.5s
991:	learn: 0.2398346	total: 39m 32s	remaining: 19.1s
992:	learn: 0.2398154	total: 39m 34s	remaining: 16.7s
993:	learn: 0.2397760	total: 39m 37s	remaining: 14.4s
994:	learn: 0.2397417	total: 39m 40s	remaining: 12s
995:	learn: 0.2397227	total: 39m 42s	remaining: 9.57s
996:	learn: 0.2396106	total: 39m 45s	remaining: 7.18s
997:	learn: 0.2395425	total: 39m 47s	remaining: 4.78s
998:	learn: 0.2395337	total: 39m 50s	remaining: 2.39s
999:	learn: 0.2394930	total: 39m 52s	remaining: 0us


In [27]:
#Trying StratifiedShuffleSplit for gridsearch with stratkfold

#didn't work

Xd,yd=train.drop('outcome_flag',axis=1),train.outcome_flag 
params = {'n_estimators': [700,1000],
        'reg_lambda' : [4,9],
        'learning_rate' : [0.07,0.13],
        'depth' : [4,7],
        'bagging_temperature' : [1]
        }                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

categorical_features_indices = np.where(Xd.dtypes =='object')[0]
catb = CatBoostRegressor(cat_features=categorical_features_indices,random_seed=13,early_stopping_rounds=100)


from sklearn.model_selection import KFold,StratifiedKFold,StratifiedShuffleSplit
SSS=StratifiedShuffleSplit(n_splits=3,test_size=0.25, random_state=13)
for train_index, test_index in SSS.split(Xd,yd):
    X_train, X_eval = Xd.iloc[train_index], Xd.iloc[test_index]
    y_train, y_eval = yd[train_index], yd[test_index]

#skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 13)
    print('start_gridsearch')
    grid = GridSearchCV(estimator=catb, param_grid=params,cv=(X_eval,y_eval),scoring='roc_auc',n_jobs=-1, verbose=100)
    print('start_fit')
    grid.fit(Xd,yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
    results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/Full_Tuning_output_Catreg_pivot_aggr_new_features_v2', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

start_gridsearch
start_fit
Fitting 2 folds for each of 16 candidates, totalling 32 fits


ValueError: too many values to unpack (expected 2)

In [30]:
#Catboost without gridsearch

from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import KFold,train_test_split


X,y=train.drop('outcome_flag',axis=1),train.outcome_flag
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 13)

categorical_features_indices = np.where(X_train.dtypes =='object')[0]


err=[]
y_pred_tot=[]
#from sklearn.model_selection import KFold,StratifiedKFold
#fold=KFold(n_splits=5,shuffle=True,random_state=1994)
#for train_index, test_index in fold.split(X,y):
#    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#    y_train, y_test = y[train_index], y[test_index]
print('start')
cat_model5 = CatBoostRegressor(n_estimators=1000, # use large n_estimators deliberately to make use of the early stopping
                         reg_lambda=4,
                         eval_metric='AUC',
                         random_seed=13,
                         learning_rate = 0.07,
                         depth = 7,
                         bagging_temperature = 1,cat_features=categorical_features_indices)
        
cat_model5.fit(X_train.values,y_train.values,eval_set=(X_val, y_val),plot=False,use_best_model=True,verbose_eval=50) # early stopping set to 100 to prevent overfitting
    #print('pred...')
    #p = cat_model5.predict(X_test)
    #if sum(p<0)>0:
    #    p=pos(p)
    #print("err: ",np.sqrt(mean_squared_error(y_test,p)))
    #err.append(np.sqrt(mean_squared_error(y_test,p)))
pred = cat_model5.predict(test)
y_pred_tot.append(pred)

start
0:	test: 0.7340467	best: 0.7340467 (0)	total: 2.02s	remaining: 33m 35s
50:	test: 0.8052854	best: 0.8052854 (50)	total: 1m 44s	remaining: 32m 15s
100:	test: 0.8135825	best: 0.8137376 (99)	total: 3m 21s	remaining: 29m 51s
150:	test: 0.8176148	best: 0.8176728 (147)	total: 5m 6s	remaining: 28m 42s
200:	test: 0.8224274	best: 0.8228970 (196)	total: 6m 51s	remaining: 27m 17s
250:	test: 0.8238379	best: 0.8243273 (237)	total: 8m 42s	remaining: 25m 57s
300:	test: 0.8254299	best: 0.8256020 (296)	total: 10m 39s	remaining: 24m 44s
350:	test: 0.8264072	best: 0.8264072 (350)	total: 12m 46s	remaining: 23m 37s
400:	test: 0.8263027	best: 0.8268363 (363)	total: 14m 41s	remaining: 21m 56s
450:	test: 0.8278504	best: 0.8278504 (450)	total: 16m 36s	remaining: 20m 13s
500:	test: 0.8288416	best: 0.8289222 (493)	total: 18m 46s	remaining: 18m 41s
550:	test: 0.8279807	best: 0.8289222 (493)	total: 20m 59s	remaining: 17m 6s
600:	test: 0.8268800	best: 0.8289222 (493)	total: 23m 9s	remaining: 15m 22s
650:	test:

In [31]:
# y_pred=rf.predict(dftest_dum.drop('id',axis=1))
print(np.mean(err,0))
y_pred=np.mean(y_pred_tot,0)
print(y_pred)

s=pd.DataFrame({'patient_id':test.patient_id,'outcome_flag':y_pred})
s.to_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Outputs/Catreg_pivot_aggr_tuned_additional_features+2cols(max spec and type)+event_flag_0.4_saved',index=False)
s.head()

nan
[0.11754776 0.57540269 0.08771293 ... 0.33450464 0.05754401 0.0795325 ]


,patient_id,outcome_flag
6,patient_10002,0.117548
7,patient_10003,0.575403
8,patient_10004,0.087713
10,patient_10006,-0.001940
16,patient_10011,-0.002795


In [62]:
test_d = pd.read_csv('C:/Users/np8022/Desktop/ZS Hack/Drug_Switch_Prediction_ParticipantsData/Inputs/test_data.csv')

test_p['patient_id']=test_d['patient_id']
#test_p.drop_duplicates(keep=False,inplace=True)
#test_p.drop_duplicates(keep=False,inplace=True)

test_p.head()

patient_id    0              patient_2
1              patien...
Name: patient_id, dtype: object

In [50]:
print(test_d[test_d['patient_id']=='patient_22686'])

            patient_id event_name  event_time specialty plan_type  \
241974   patient_22686   event_22         805    spec_1    plan_2   
382452   patient_22686   event_37         805    spec_1    plan_2   
540472   patient_22686   event_37         805    spec_1    plan_2   
750247   patient_22686   event_22         805    spec_1    plan_2   
797473   patient_22686   event_37         805    spec_1    plan_2   
839707   patient_22686  event_104         805    spec_1    plan_2   
904823   patient_22686   event_98         805    spec_1    plan_2   
972863   patient_22686   event_22         805    spec_1    plan_2   
1049810  patient_22686   event_22         805    spec_1    plan_2   
1144833  patient_22686   event_37         805    spec_1    plan_2   
1150853  patient_22686  event_104         805    spec_1    plan_2   
1310404  patient_22686   event_98         805    spec_1    plan_2   
1488764  patient_22686   event_98         805    spec_1    plan_2   
1570966  patient_22686  event_104 